<a href="https://colab.research.google.com/github/secutron/ShowTime/blob/master/dp0005.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 이거슨...



In [2]:
# Tensorboard 준비작업

LOG_DIR = 'drive/data/tb_logs'
	
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
	
import os
if not os.path.exists(LOG_DIR):
	os.makedirs(LOG_DIR)
	  
get_ipython().system_raw(
	'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
	.format(LOG_DIR))
	
get_ipython().system_raw('./ngrok http 6006 &')
	
!curl -s http://localhost:4040/api/tunnels | python3 -c \
	"import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2019-09-11 01:29:51--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.22.145.207, 52.3.53.115, 52.201.75.180, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.22.145.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13607069 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  12.98M  76.2MB/s    in 0.2s    

2019-09-11 01:29:51 (76.2 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13607069/13607069]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
https://801fbe9a.ngrok.io


In [3]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.callbacks import TensorBoard

batch_size = 128
num_classes = 10
epochs = 12
	
### input image dimensions
img_rows, img_cols = 28, 28
	
### the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
	
if K.image_data_format() == 'channels_first':
	x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
	x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
	input_shape = (1, img_rows, img_cols)
else:
	x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
	x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
	input_shape = (img_rows, img_cols, 1)
	
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
	
### convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
	
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
	activation='relu',
	input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
	
model.compile(loss=keras.losses.categorical_crossentropy,
	optimizer=keras.optimizers.Adadelta(),
	metrics=['accuracy'])
	
	
tbCallBack = TensorBoard(log_dir=LOG_DIR, 
	histogram_freq=1,
	write_graph=True,
	write_grads=True,
	batch_size=batch_size,
	write_images=True)
	
model.fit(x_train, y_train,
	batch_size=batch_size,
	epochs=epochs,
	verbose=1,
	validation_data=(x_test, y_test),
	callbacks=[tbCallBack])
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Using TensorFlow backend.


11493376/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 60000 samples, validate on 10000 samples




Epoch 1/12
60000/60000 [==============================] - 156s 3ms/step - loss: 0.2617 - acc: 0.9204 - val_loss: 0.0563 - val_acc: 0.9823
Epoch 2/12
60000/60000 [==============================] - 155s 3ms/step - loss: 0.0869 - acc: 0.9746 - val_loss: 0.0398 - val_acc: 0.9871
Epoch 3/12
60000/60000 [==============================] - 156s 3ms/step - loss: 0.0645 - acc: 0.9809 - val_loss: 0.0318 - val_acc: 0.9892
Epoch 4/12
60000/60000 [==============================] - 156s 3ms/step - loss: 0.0517 - acc: 0.9835 - val_loss: 0.0283 - val_acc: 0.9905
Epoch 5/12
60000/60000 [================